# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 9:39AM,258028,19,TR-RN-02242023,"GCH Granules USA, Inc.",Released
1,Feb 23 2023 9:37AM,257983,15,9192335,"Mizner Bioscience, LLC",Released
2,Feb 23 2023 9:37AM,257983,15,9192336,"Mizner Bioscience, LLC",Released
3,Feb 23 2023 9:36AM,257943,15,VT80184370,"Virtus Pharmaceuticals, LLC",Released
4,Feb 23 2023 9:35AM,257986,15,PNC429248A,"Person & Covey, Inc.",Released
5,Feb 23 2023 9:35AM,257986,15,PNC429250A,"Person & Covey, Inc.",Released
6,Feb 23 2023 9:35AM,258021,10,PRF-53853,Bio-PRF,Released
7,Feb 23 2023 9:35AM,258021,10,PRF-53864,Bio-PRF,Released
8,Feb 23 2023 9:35AM,258021,10,PRF-53869,Bio-PRF,Released
9,Feb 23 2023 9:35AM,258021,10,PRF-53872,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,258022,Released,57
40,258023,Released,7
41,258024,Released,1
42,258025,Released,5
43,258028,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258022,NaN,NaN,57.0
258023,NaN,NaN,7.0
258024,NaN,NaN,1.0
258025,NaN,NaN,5.0
258028,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0.0,0.0,13.0
257934,0.0,0.0,2.0
257935,2.0,3.0,12.0
257936,0.0,0.0,23.0
257942,0.0,0.0,25.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257888,0,0,13
257934,0,0,2
257935,2,3,12
257936,0,0,23
257942,0,0,25


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,0,0,13
1,257934,0,0,2
2,257935,2,3,12
3,257936,0,0,23
4,257942,0,0,25


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257888,,,13
1,257934,,,2
2,257935,2,3,12
3,257936,,,23
4,257942,,,25


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc."
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC"
3,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC"
4,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc."
6,Feb 23 2023 9:35AM,258021,10,Bio-PRF
25,Feb 23 2023 9:35AM,257888,15,"Person & Covey, Inc."
38,Feb 23 2023 9:34AM,257985,15,"Alliance Pharma, Inc."
39,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation
96,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc."
121,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc.",,,1
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",,,2
2,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC",,,1
3,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc.",,,2
4,Feb 23 2023 9:35AM,258021,10,Bio-PRF,,,19
5,Feb 23 2023 9:35AM,257888,15,"Person & Covey, Inc.",,,13
6,Feb 23 2023 9:34AM,257985,15,"Alliance Pharma, Inc.",,,1
7,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,,,57
8,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",,,25
9,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",,,23


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc.",1,,
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",2,,
2,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC",1,,
3,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc.",2,,
4,Feb 23 2023 9:35AM,258021,10,Bio-PRF,19,,
5,Feb 23 2023 9:35AM,257888,15,"Person & Covey, Inc.",13,,
6,Feb 23 2023 9:34AM,257985,15,"Alliance Pharma, Inc.",1,,
7,Feb 23 2023 9:33AM,258022,10,ISDIN Corporation,57,,
8,Feb 23 2023 9:32AM,257942,15,"Alliance Pharma, Inc.",25,,
9,Feb 23 2023 9:30AM,257936,15,"Alliance Pharma, Inc.",23,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc.",1,,
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",2,,
2,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC",1,,
3,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc.",2,,
4,Feb 23 2023 9:35AM,258021,10,Bio-PRF,19,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",2.0,NaN,NaN
2,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC",1.0,NaN,NaN
3,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc.",2.0,NaN,NaN
4,Feb 23 2023 9:35AM,258021,10,Bio-PRF,19.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 9:39AM,258028,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
1,Feb 23 2023 9:37AM,257983,15,"Mizner Bioscience, LLC",2.0,0.0,0.0
2,Feb 23 2023 9:36AM,257943,15,"Virtus Pharmaceuticals, LLC",1.0,0.0,0.0
3,Feb 23 2023 9:35AM,257986,15,"Person & Covey, Inc.",2.0,0.0,0.0
4,Feb 23 2023 9:35AM,258021,10,Bio-PRF,19.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5933795,273.0,9.0,20.0
12,258017,1.0,0.0,0.0
15,1805663,67.0,0.0,0.0
16,516034,2.0,0.0,0.0
18,258009,1.0,0.0,0.0
19,516029,24.0,13.0,0.0
21,773891,2.0,1.0,0.0
22,515995,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5933795,273.0,9.0,20.0
1,12,258017,1.0,0.0,0.0
2,15,1805663,67.0,0.0,0.0
3,16,516034,2.0,0.0,0.0
4,18,258009,1.0,0.0,0.0
5,19,516029,24.0,13.0,0.0
6,21,773891,2.0,1.0,0.0
7,22,515995,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,273.0,9.0,20.0
1,12,1.0,0.0,0.0
2,15,67.0,0.0,0.0
3,16,2.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,24.0,13.0,0.0
6,21,2.0,1.0,0.0
7,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,273.0
1,12,Released,1.0
2,15,Released,67.0
3,16,Released,2.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21,22
Status,,,,,,,,
Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,0.0,0.0,0.0,0.0,13.0,1.0,1.0
Released,273.0,1.0,67.0,2.0,1.0,24.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,0.0,0.0,13.0,1.0,1.0
2,Released,273.0,1.0,67.0,2.0,1.0,24.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,0.0,0.0,0.0,0.0,13.0,1.0,1.0
2,Released,273.0,1.0,67.0,2.0,1.0,24.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()